In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import json
import requests
from pymongo import MongoClient
import numpy as np
from collections import defaultdict
import pickle
import networkx as nx

In [2]:
from scrape_util import (load_json,
                        write_json,
                        selenium_instagram_login,
                        setup_mongo_client)


In [3]:
client, collection = setup_mongo_client('instascrape', 'influencer_likes')

# Making dictionary of top posts for top influencers

In [5]:
hashtags_likes_dict = load_json('../data/hashtags_likes_dict.json')
top = nx.read_gml('../data/graph_withSelenaGomez_reduced.gml')

In [8]:
# Collect posts for all top influencers
influencers_top = {}
for k,v in hashtags_likes_dict.iteritems():
    if k in top.nodes():
        influencers_top[k] = v

In [10]:
# Collect most liked post for all top influencers
influencers_top_post = {}
for k,v in influencers_top.iteritems():
    influencers_top_post[k] = {'max_likes':0, 'post': None}
    for i in range(len(v['likes'])):
        if v['likes'][i] > influencers_top_post[k]['max_likes']:
            influencers_top_post[k]['max_likes'] = v['likes'][i]
            influencers_top_post[k]['post'] = v['posts'][i]

In [21]:
top = influencers_top_post.keys()

# SCRAPE THE REST!

In [23]:
driver = webdriver.Firefox()
url_likes = "https://www.instagram.com/graphql/query/?query_id=17864450716183058&variables=%7B%22shortcode%22%3A%22{}%22%2C%22first%22%3A{}%7D"
selenium_instagram_login(driver, '../instagram_credentials.json')

In [32]:
for i in range(len(top)):
    driver.get(url_likes.format(influencers_top_post[top[i]]['post'], str(influencers_top_post[top[i]]['max_likes'])))
    time.sleep(np.random.uniform(3,5))
    
    driver.find_element_by_id('tab-1').click()
    data = driver.find_element_by_css_selector('pre.data')
    my_json = json.loads(data.text)
    likes = my_json['data']['shortcode_media']['edge_liked_by']['edges']
    collection.insert_one({'id':top[i],'likes': likes})
    
    time.sleep(np.random.uniform(5,8))

# Saving the collected data

In [41]:
cursor = collection.find({})
for x in cursor:
    user_id = x['id']
    likes = []
    for liker in x['likes']:
        likes.append(liker['node']['id'])
    if influencers_top_post.get(user_id, False):
        influencers_top_post[user_id]['likes'] = likes

## Add total followers

In [42]:
G = nx.read_gml('../data/graph_withSelenaGomez.gml')

In [43]:
for influencer in influencers_top_post:
    influencers_top_post[influencer]['num_follow'] = len(G.in_edges(influencer))

In [46]:
write_json(influencers_top_post, '../data/interaction_dict.json')

# Checking the data

In [48]:
for k,v in influencers_top_post.iteritems():
    if 'likes' in v.keys():
        print k, v['max_likes'], len(v['likes'])

 294756617 885 885
859178340 236 236
5090616 3159 3159
176532308 312 312
2170373061 270 270
401312409 170 170
1982904005 462 462
400512410 284 284
55514310 2270 2268
28898672 268 268
481991581 2298 2296
3116828063 643 643
1257079395 3928 3928
1069513983 675 675
290716168 1222 1222
1495667341 4619 4619
1108148509 52 52
4562782 184 184
4808210637 620 620
1643549487 139 139
252464337 277 277
414679969 232 232
2252884188 1486 1486
1313836206 157 157
11295358 5330 5330
205770677 121 121
11454662 812 811
407812568 199 199
29333708 314 314
18891963 680 680
53274987 92 92
235439673 722 722
1948983829 631 631
356143 193 193
223936626 610 610
2218921 808 808
25647980 1359 1359
458602313 2142 2142
6677416 66 66
20340822 681 681
3466746620 736 736
269335057 224 224
555987737 677 677
53440747 1434 1434
315614120 1115 1115
270129251 568 568
3933407045 377 377
146154834 362 362
2283923757 250 250
3935144838 1597 1596
47296290 3158 3158
200605625 760 760
1977468026 3647 3647
1747270904 445 445
4056371